<a href="https://colab.research.google.com/github/Aipakazuma/meetup037/blob/master/005_Functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functional API

応用がきくAPIを使って、モデルのカスタマイズを扱いやすくする

## やること

* 簡単なFunctional API
* Inceptionモデルを組んでみる

In [0]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.callbacks import TensorBoard


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

Using TensorFlow backend.


## データの読み込み

In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

## 前処理

In [0]:
# 画像を0-1の範囲で正規化
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# 正解ラベルをOne-Hot表現に変換
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

## Functional APIで簡単なモデルを構築する

## モデルの構築

In [0]:
from keras.models import Input, Model
from keras.layers import Dense, Flatten, Activation

In [0]:
# 入力する層と出力する層をきめてあげて、あとは順番にレイヤーを組んでいけばおｋ
# 入力層
_input = Input(shape=(32,32,3))

# 中間層
x = Flatten()(_input)

x = Dense(32)(x)
x = Activation('relu')(x)

x = Dense(64)(x)
x = Activation('relu')(x)

# 出力層
_output = Dense(10, activation='softmax')(x)

# モデルにする
model = Model(_input, _output)

# compileは一緒
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                98336     
_________________________________________________________________
activation_5 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                2112      
_________________________________________________________________
activation_6 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total para

In [0]:
epochs = 20
batch_size = 128
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                  validation_data=(X_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 5s 95us/step - loss: 1.9805 - acc: 0.2657 - val_loss: 1.8604 - val_acc: 0.3248
Epoch 2/20
50000/50000 [==============================] - 4s 78us/step - loss: 1.8329 - acc: 0.3360 - val_loss: 1.8017 - val_acc: 0.3514
Epoch 3/20
50000/50000 [==============================] - 4s 78us/step - loss: 1.7974 - acc: 0.3499 - val_loss: 1.7743 - val_acc: 0.3600
Epoch 4/20
50000/50000 [==============================] - 4s 77us/step - loss: 1.7756 - acc: 0.3599 - val_loss: 1.7597 - val_acc: 0.3545
Epoch 5/20
50000/50000 [==============================] - 4s 78us/step - loss: 1.7574 - acc: 0.3662 - val_loss: 1.7715 - val_acc: 0.3558
Epoch 6/20
50000/50000 [==============================] - 4s 79us/step - loss: 1.7476 - acc: 0.3708 - val_loss: 1.7914 - val_acc: 0.3553
Epoch 7/20
50000/50000 [==============================] - 4s 78us/step - loss: 1.7405 - acc: 0.3738 - val_loss: 1.7376 - val_acc

## Inseptionモデルの構築


さぁ組み込もうと思ったら

https://keras.io/ja/applications/#inceptionv3

なんとすでに学習済みモデルが...(今回使わないけど)

### Inseption v3とは

>Googleによって開発されたInception-v3は、ILSVRCという大規模画像データセットを使った画像識別タスク用に1,000クラスの画像分類を行うよう学習されたモデルで、非常に高い精度の画像識別を達成しています。

<img src="https://vitalab.github.io/deep-learning/images/inception/fig_5.png">

In [0]:
from keras.layers import Dense, concatenate
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D


# 入力する層と出力する層をきめてあげて、あとは順番にレイヤーを組んでいけばおｋ
# 入力層
_input = Input(shape=(32,32,3))

# 中間層
# block1
x = Conv2D(128, (1,1), strides=4)(_input)
block1 = Activation('relu')(x)

# block2
x = AveragePooling2D(3, strides=2)(_input)
x = Conv2D(128, (1,1), strides=2)(x)
block2 = Activation('relu')(x)

# block3
x = Conv2D(128, (1,1), strides=2)(_input)
x = Activation('relu')(x)
x = Conv2D(128, (2,2), strides=2)(x)
block3 = Activation('relu')(x)

# block4
x = Conv2D(128, (1,1))(_input)
x = Activation('relu')(x)
x = Conv2D(128, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3,3), strides=4)(x)
block4 = Activation('relu')(x)

# 連結
con = concatenate([block1,block2,block3,block4], axis=-1)
x = Flatten()(con)

# 出力層
_output = Dense(10, activation='softmax')(x)

# モデルにする
model = Model(_input, _output)

# compileは一緒
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_68 (Conv2D)              (None, 32, 32, 128)  512         input_13[0][0]                   
__________________________________________________________________________________________________
activation_74 (Activation)      (None, 32, 32, 128)  0           conv2d_68[0][0]                  
__________________________________________________________________________________________________
conv2d_66 (Conv2D)              (None, 16, 16, 128)  512         input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_69 

In [0]:
epochs = 20
batch_size = 128
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                                  validation_data=(X_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 33s 650us/step - loss: 1.5370 - acc: 0.4543 - val_loss: 1.3092 - val_acc: 0.5303
Epoch 2/20
50000/50000 [==============================] - 30s 593us/step - loss: 1.1635 - acc: 0.5910 - val_loss: 1.1185 - val_acc: 0.6057
Epoch 3/20
50000/50000 [==============================] - 30s 595us/step - loss: 1.0049 - acc: 0.6504 - val_loss: 1.0844 - val_acc: 0.6212
Epoch 4/20
50000/50000 [==============================] - 30s 595us/step - loss: 0.8943 - acc: 0.6896 - val_loss: 1.0300 - val_acc: 0.6425
Epoch 5/20
50000/50000 [==============================] - 30s 594us/step - loss: 0.8005 - acc: 0.7218 - val_loss: 0.9906 - val_acc: 0.6584
Epoch 6/20
50000/50000 [==============================] - 30s 595us/step - loss: 0.7169 - acc: 0.7509 - val_loss: 1.0493 - val_acc: 0.6495
Epoch 7/20
50000/50000 [==============================] - 30s 595us/step - loss: 0.6406 - acc: 0.7786 - val_loss: 1.

## 多出力モデル

さっきのモデルから出力する層を１つ追加してみましょう -> 入力された画像から2の出力がえられる

In [0]:
from keras.layers import Dense, concatenate
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D


# 入力する層と出力する層をきめてあげて、あとは順番にレイヤーを組んでいけばおｋ
# 入力層
_input = Input(shape=(32,32,3))

# 中間層
# block1
x = Conv2D(128, (1,1), strides=4)(_input)
block1 = Activation('relu')(x)

# block2
x = AveragePooling2D(3, strides=2)(_input)
x = Conv2D(128, (1,1), strides=2)(x)
block2 = Activation('relu')(x)

# block3
x = Conv2D(128, (1,1), strides=2)(_input)
x = Activation('relu')(x)
x = Conv2D(128, (2,2), strides=2)(x)
block3 = Activation('relu')(x)

# block4
x = Conv2D(128, (1,1))(_input)
x = Activation('relu')(x)
x = Conv2D(128, (3,3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3,3), strides=4)(x)
block4 = Activation('relu')(x)

# 連結
con = concatenate([block1,block2,block3,block4], axis=-1)
x = Flatten()(con)

# 出力層1
# あとから損失関数と紐付けるためにレイヤーに名前をつける
_output1 = Dense(10, activation='softmax', name='output1')(x)


# block4から全結合層2つならべて出力するblock(枝？)を用意する
x = Flatten()(block4)
x = Dense(128)(x)
x = Activation('relu')(x)
x = Dense(64)(x)
x = Activation('relu')(x)
_output2 = Dense(10, activation='softmax', name='output2')(x)

# モデルにする(outputが2つあることに注意)
model = Model(_input, [_output1, _output2])

# compileは複数の損失を指定してあげる
model.compile(optimizer='adam',loss={'output1': 'categorical_crossentropy', 'output2': 'categorical_crossentropy'}, metrics=['accuracy'])

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_82 (Conv2D)              (None, 32, 32, 128)  512         input_15[0][0]                   
__________________________________________________________________________________________________
activation_90 (Activation)      (None, 32, 32, 128)  0           conv2d_82[0][0]                  
__________________________________________________________________________________________________
conv2d_83 (Conv2D)              (None, 32, 32, 128)  147584      activation_90[0][0]              
__________________________________________________________________________________________________
activation

In [0]:
epochs = 20
batch_size = 128

# 出力が２つになったのでdictで渡します
history = model.fit(X_train, {'output1': y_train, 'output2':y_train}, epochs=epochs, batch_size=batch_size,
                                  validation_data=(X_test, {'output1': y_test, 'output2':y_test}))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 33s 664us/step - loss: 3.1440 - output1_loss: 1.5473 - output2_loss: 1.5967 - output1_acc: 0.4516 - output2_acc: 0.4245 - val_loss: 2.6561 - val_output1_loss: 1.3039 - val_output2_loss: 1.3522 - val_output1_acc: 0.5334 - val_output2_acc: 0.5147
Epoch 2/20
50000/50000 [==============================] - 32s 636us/step - loss: 2.4685 - output1_loss: 1.2350 - output2_loss: 1.2335 - output1_acc: 0.5651 - output2_acc: 0.5580 - val_loss: 2.3645 - val_output1_loss: 1.2030 - val_output2_loss: 1.1615 - val_output1_acc: 0.5668 - val_output2_acc: 0.5829
Epoch 3/20
50000/50000 [==============================] - 32s 637us/step - loss: 2.1701 - output1_loss: 1.1033 - output2_loss: 1.0668 - output1_acc: 0.6124 - output2_acc: 0.6239 - val_loss: 2.2344 - val_output1_loss: 1.1314 - val_output2_loss: 1.1030 - val_output1_acc: 0.6010 - val_output2_acc: 0.6068
Epoch 4/20
50000/50000 [=================